In [1]:
# Online Appendix for the Honors Thesis in Mathematics -  Notebook 15
# Created on 18 November 2021
# Author: Batu El

In [2]:
# import statements
import numpy as np
######################################
# Basic Functions 1: Length Functions#
######################################
# Function 1: Length of a Permutation
def length(perm):
    # input: a fixed-point-free involution of n numbers (1,2,..,n)
    # output: integer that denotes the length as a permutation, i.e. number of pairs of elements that are out of order
    perm_copy = perm.copy()
    length = 0
    n = len(perm_copy)
    for i in range(n):
        for j in range(i+1,n):
            if perm_copy[i] > perm_copy[j]:
                length += 1
    return length
# Function 2: Converts FPF inv to oneline noataion
def fpfinv_to_oneline(pi ):
    # input: a fixed-point-free involution in cycle notation
    # output: same fixed-point-free involution in one-line notation
    l = len(pi)
    a = list(range(1, l+1))
    for i in range(len(pi)):
        pos = pi[i]-1
        if i%2 ==0:
            val = pi[i+1]
        if i%2 != 0:
            val = pi[i-1]
        a[pos] = val
    return a
# Function 3: Calculates FPF-Length
def fpf_lenght(pi):
    # input: a fixed-point-free involution of n numbers (1,2,..,n) in cycle notation
    # output: integer or float, denotes lenth as a fpf involution
    n = len(pi)
    oneline_length = length(fpfinv_to_oneline(pi))
    num_2cycles = (n/2)
    pi_length = (oneline_length - num_2cycles)/2
    return pi_length
#####################################################
# Basic Functions 2: Simple Transposition Functions #
#####################################################
# Function 4: simpleTransposition
def simpleTransposition(perm, i):
    # input: permutation, transposition
    # output: transposed permutation
    transposed_perm = perm.copy()
    a = perm[i-1]
    b = perm[i]
    transposed_perm[i-1] = b
    transposed_perm[i] = a        
    return transposed_perm
# Function 5: multipleTransposition
def multipleTransposition(perm, path):
    # input: permutation, list of transpositions
    # output: transposed permutation
    perm_copy = perm.copy()
    for i in path:
        perm_copy = simpleTransposition(perm_copy, i)
    return perm_copy
############################################################
# Class for Generalized Fixed Point Free Involution Object #
############################################################
class muInvolutionNode:
    " a node muInvolutionNode in Hasse diagram "
    # mu is a list of cycles, example: [2,4] for n=6
    # w_prime is the notation of the permutation
    def __init__(self, mu, w_prime , parts_cycle_notation_list):
        # initializer
        self.mu = mu
        self.w_prime = w_prime
        self.parts_cycle_notation_list = parts_cycle_notation_list
    def w(self):
        w = []
        currentPartStart = 0
        currentPartEnd = 0
        for i in range(len(self.mu)):
            currentPartEnd += self.mu[i]
            currentPart = self.w_prime[currentPartStart:currentPartEnd]
            aux_list = np.zeros_like(currentPart)
            for j in range(len(aux_list)):
                aux_list[j] = currentPart[self.parts_cycle_notation_list[i][j] - 1]
            w.append(list(aux_list))
            currentPartStart += self.mu[i]
        return w
    def muLength(self):
        return mu_length(self)
    def __repr__(self):
        # for formatted printing
        string = ''
        for i in self.w():
            string += str(i)[1:-1]
            string += ' | '
        string = string[:-2] # eliminate redundant '|'
        string = string.replace(" ", "")
        string = string.replace(",", " ")
        return '[' + string + ']'
        #returns '--mu:'+ str(self.mu) +' w_prime:'+str(self.w_prime)+' parts_cycle_notation_list:'+str(self.parts_cycle_notation_list)+"--"
    def muSimpleTransposition(self, transposition_index):
        return simple_transposition(self , transposition_index)
    def muSimpleTransposition_nc(self, transposition_index):
        return simple_transposition_no_constraint(self , transposition_index)   
###############################################################
# Functions For Generalized FPF Involution 1: Length Function # 
###############################################################
# Function 6: mu_length
def mu_length( node ):
    # w_prime length + sum of fpf lengths
    w_prime_length = length(node.w_prime)
    sum_ = 0
    for i in range(len(node.parts_cycle_notation_list)):
        sum_ += fpf_lenght(node.parts_cycle_notation_list[i])
    sum_ += w_prime_length
    return sum_
##########################################################################################################
# Functions For Generalized FPF Involution 2: Helper Functions to Define Generalized Simple Transpoition # 
##########################################################################################################
# Function 7: samePart
def samePart(node , transposition_index):
    # note :   1 <= transposition_index <= n-1
    transposition_index_1 = transposition_index
    transposition_index_2 = transposition_index + 1
    # tests if i and i+1 are in the same part
    samePart = False # boolean value
    currentPartStart = 0
    currentPartEnd = 0
    for i in range(len(node.mu)):
        currentPartEnd += node.mu[i]
        for j in range(len(node.w_prime)):
            currentPart = node.w_prime[currentPartStart:currentPartEnd]
            if transposition_index_1 in currentPart:
                if transposition_index_2 in currentPart:
                    samePart = True
                    return (samePart, i, node.w_prime[currentPartStart:currentPartEnd])
        currentPartStart += node.mu[i]
    # returns ( boolean , index of the part , the elements in the part)
    return (samePart, -1, [])
# Function 8: change_by_index
def change_by_index(blist, a_index , b_index):
    # changes the places of elements in index a and index b
    clist = blist.copy()
    c = clist[a_index]
    d = clist[b_index]
    clist[a_index] = d
    clist[b_index] = c
    return clist
# Function 9: change_if_length_increases
def change_if_length_increases(alist, a , b):
    # changes the elements in position i (index i-1) and  and position i+1 (index i) if length increases
    blist = alist.copy()
    a_index = 0
    b_index = 0
    for i  in range(len(alist)):
        if alist[i] == a:
            a_index = i
        if alist[i] == b:
            b_index = i
    transposed = change_by_index(alist , a_index, b_index)
    if length(transposed) > length(alist):
        return transposed
    else:
        return alist
# Function 10: change_with_no_constraint
def change_with_no_constraint(alist, a , b):
    # changes the elements in position i (index i-1) and  and position i+1 (index i) if length increases
    blist = alist.copy()
    a_index = 0
    b_index = 0
    for i  in range(len(alist)):
        if alist[i] == a:
            a_index = i
        if alist[i] == b:
            b_index = i
    transposed = change_by_index(alist , a_index, b_index)
    return transposed
# Function 11: fpf_transform
def fpf_transform(alist, a , b):
    # IMPORTANT: changes positions of numbers a and b; does not change numbers at position a and position b
    blist = alist.copy()
    # a_index and b_index are the positions of the numbers 
    a_index = 0
    b_index = 0
    for i  in range(len(alist)):
        if alist[i] == a:
            a_index = i
        if alist[i] == b:
            b_index = i
    # Case1: if a_index is even and b_index is even
    # Case2: if a_index is even and b_index is odd
    # Case3: if a_index is odd and b_index is even
    # Case4: if a_index is odd and b_index is odd
    case = 0
    if a_index%2 == 0:
        if b_index%2 == 0:
            case = 1
        if b_index%2 != 0:
            case = 2
    if a_index%2 != 0:
        if b_index%2 == 0:
            case = 3
        if b_index%2 != 0:
            case = 4
    if case ==1:
        transposed = change_by_index(alist , a_index+1, b_index+1)
    if case ==2:
        if a_index+1 == b_index:
            transposed = alist
        else:
            transposed = change_by_index(alist , a_index, b_index)
    if case ==3:
        if b_index+1 == a_index:
            transposed = alist
        else:
            transposed = change_by_index(alist , a_index, b_index)
    if case ==4:
        transposed = change_by_index(alist , a_index, b_index)
    return transposed
# Function 12: change_if_fpf_length_increases
def change_if_fpf_length_increases(alist, a , b):
    # changes the elements in position i (index i-1) and  and position i+1 (index i) if length increases
    transposed = fpf_transform(alist , a, b)
    if fpf_lenght(transposed) > fpf_lenght(alist):
        return transposed
    else:
        return alist
# Function 13: transform_transformation
def transform_transformation(transposition_index_1 , transposition_index_2 , part_w_prime):
    # we need to transform the transformation before we apply it    
    transposition_index = 0
    for i in range(len(part_w_prime)):
        if part_w_prime[i] == transposition_index_1:
            transposition_index = i
    transformed_transposition_index_1 = transposition_index + 1
    transposition_index = 0
    for i in range(len(part_w_prime)):
        if part_w_prime[i] == transposition_index_2:
            transposition_index = i        
    transformed_transposition_index_2 = transposition_index + 1
    return transformed_transposition_index_1 , transformed_transposition_index_2
###############################################################################
# Functions For Generalized FPF Involution 3: Generalized Simple Transpoition # 
###############################################################################
# Function 14: simple_transposition
def simple_transposition( node , transposition_index ): # only increases the length
    # note :   1 <= transposition_index <= n-1
    transposition_index_1 = transposition_index
    transposition_index_2 = transposition_index + 1
    samePart_ = samePart(node , transposition_index)
    samePart_bool = samePart_[0] #True if in same part
    if samePart_bool: #when two indices for transpostion are in the same part
        # find relevant part
        samePart_index = samePart_[1] # index of the part, -1 if not in same part
        part_w_prime = samePart_[2] # the part itself, ex: [3,4,5,6]
        part_cycle_notation = node.parts_cycle_notation_list[samePart_index] # ex: [1,2,3,4]
        transformed_transposition_index_1 , transformed_transposition_index_2 = transform_transformation(transposition_index_1 , transposition_index_2 , part_w_prime)
        # calculate new part
        new_part_cycle_notation = change_if_fpf_length_increases(part_cycle_notation, transformed_transposition_index_1 , transformed_transposition_index_2)
        # keep all the other parts constant
        new_parts_cycle_notation_list = node.parts_cycle_notation_list.copy()
        # change the relevant part with new relevant part
        new_parts_cycle_notation_list[samePart_index] = new_part_cycle_notation
        return muInvolutionNode(node.mu, node.w_prime, new_parts_cycle_notation_list)
    else: #when two indices for transpostion are not in the same part
        # change the places of the indices n w_prime
        new_w_prime = change_if_length_increases(node.w_prime, transposition_index_1, transposition_index_2)
        return muInvolutionNode(node.mu, new_w_prime, node.parts_cycle_notation_list)
# Function 15: simple_transposition_no_constraint
def simple_transposition_no_constraint( node , transposition_index ): # can decrease length
    # note :   1 <= transposition_index <= n-1
    transposition_index_1 = transposition_index
    transposition_index_2 = transposition_index + 1
    samePart_ = samePart(node , transposition_index)
    samePart_bool = samePart_[0] #True if in same part
    if samePart_bool: #when two indices for transpostion are in the same part
        # find relevant part
        samePart_index = samePart_[1] # index of the part, -1 if not in same part
        part_w_prime = samePart_[2] # the part itself, ex: [3,4,5,6]
        part_cycle_notation = node.parts_cycle_notation_list[samePart_index] # ex: [1,2,3,4]
        transformed_transposition_index_1 , transformed_transposition_index_2 = transform_transformation(transposition_index_1 , transposition_index_2 , part_w_prime)
        # calculate new part
        ### change this line to eliminate line increase constraint
        new_part_cycle_notation = fpf_transform(part_cycle_notation, transformed_transposition_index_1 , transformed_transposition_index_2)
        # keep all the other parts constant
        new_parts_cycle_notation_list = node.parts_cycle_notation_list.copy()
        # change the relevant part with new relevant part
        new_parts_cycle_notation_list[samePart_index] = new_part_cycle_notation
        return muInvolutionNode(node.mu, node.w_prime, new_parts_cycle_notation_list)
    else: #when two indices for transpostion are not in the same part
        # change the places of the indices n w_prime
        new_w_prime = change_with_no_constraint(node.w_prime, transposition_index_1, transposition_index_2)
        return muInvolutionNode(node.mu, new_w_prime, node.parts_cycle_notation_list)

#######################################################################
# Functions For Calculating the Chains in Generalized FPF Involutions # 
#######################################################################
# Function 16: simple_transposition_no_constraint
def calculateMaximumLength(initial_node):
    simpleTranspositions = list(range(1,sum(initial_node.mu)))
    maxNode = initial_node
    max_length = 0
    while True:
        break_now = True
        for i in simpleTranspositions:
            current_node = maxNode.muSimpleTransposition(i)
            current_length = current_node.muLength()
            if current_length > max_length:
                max_length = current_length
                maxNode = current_node
                break_now = False
        if break_now:
            break
    return (max_length , maxNode )
# Function 17: apply_multiple_transpositions
def apply_multiple_transpositions( node , trasposition_list):
    # apply multiple transpositions to a node
    current_node_base = node
    for transp in trasposition_list:
        current_node_base = current_node_base.muSimpleTransposition(transp)    
    return  current_node_base
# Function 18: generate_nodes_given_paths
def generate_nodes_given_paths( node , paths_list ):
    # apply multiple transposiitions to a node
    preserve_initial = node
    nodes_given_paths = []
    for path in paths_list:
        current_node = apply_multiple_transpositions(preserve_initial , path)
        nodes_given_paths.append(current_node)
    return nodes_given_paths
# Function 19: incoming_edges
def incoming_edges( node ):
    initial_length = node.muLength()
    simpleTranspositions = list(range(1,sum(node.mu)))
    incoming = []
    for i in simpleTranspositions:
        current_node = node.muSimpleTransposition_nc(i)
        current_node_length = current_node.muLength()
        if current_node_length == initial_length-1:
            incoming.append((i , current_node))
    return incoming
# Function 20: outgoing_edges
def outgoing_edges( node ):
    initial_length = node.muLength()
    simpleTranspositions = list(range(1,sum(node.mu)))
    outgoing = []
    for i in simpleTranspositions:
        current_node = node.muSimpleTransposition_nc(i)
        current_node_length = current_node.muLength()
        if current_node_length == initial_length+1:
            outgoing.append((i , current_node))
    return outgoing
# Function 22: apply_trasp_to_w
def apply_trasp_to_w(w_set , transp ):
    new_w_set = set({})
    for e in w_set:
        e = list(e)
        new_e = simpleTransposition(e, transp)
        new_e = tuple(new_e)
        new_w_set.add(new_e)
    return new_w_set
# Function 23: mu_node_equality
def mu_node_equality( node1 ,  node2):
    a = node1.mu == node2.mu
    b = node1.w_prime == node2.w_prime
    c = node1.parts_cycle_notation_list == node2.parts_cycle_notation_list
    return a and b and c
# Function 24: mu_node_equality
def invert_w_set(w_set):
    new_w_set = set({})
    for current in w_set:
        length = len(current)
        a = list(np.zeros([length]))
        for i in range(length):
            a[current[i]-1] = int(i+1)
        a = set({tuple(a)})
        new_w_set = new_w_set.union(a)
    return new_w_set
#################################################################################################
# How to use these functions  ?                                                                 #
#################################################################################################
# initial_node = muInvolutionNode([2,4], [1,2,3,4,5,6], [[1,2],[1,2,3,4]])                      #
#  apply_multiple_transpositions(initial_node , [2,1])                                          #
#  a = generate_nodes_given_paths(initial_node , [[2,3,4,5,3] , [2,3,4,5,3,4], [2,3,4,5,3,2]])  #
# node0 = muInvolutionNode([2,4], [5,6,1,2,3,4], [[1,2],[1,4,2,3]])                             #
# print(incoming_edges(node0))                                                                  #
# print(outgoing_edges(node0))                                                                  #
# print(apply_trasp_to_w({(1,2,3,4,5,6) , (1,3,2,4,5,6)} , 1))                                  #
# ###############################################################################################

#######################################################################################
# Class for the Tree Associated with a Generalized Fixed Point Free Involution Object #
#######################################################################################
class inv_Tree:
    def __init__(self , initial_node):
        self.root = initial_node
        self.length , self.leaf = calculateMaximumLength(initial_node)
        self.length = int(self.length)
    def invert_w_sets(self):
        dictionary = self.tree_dict
        for i in range(len(dictionary.keys())):
            current_key = list(dictionary.keys())[i]
            current_level = dictionary[current_key]
            for j in range(len(current_level)):
                self.tree_dict[current_key][j][2] = invert_w_set(self.tree_dict[current_key][j][2])
    def construct_dictionary(self):
        keys_list = []
        for i in range(self.length+1):
            keys_list.append(str(i))
        print('number of levels:' , int(keys_list[-1]))
        # initialize the dictionary
        tree_dict = dict({})
        # Add the initial node manually to the dictionary
        current_node = self.root
        current_w_set = set({tuple(range(1,sum(self.root.mu)+1))})
        current_tuple = [current_node , outgoing_edges( current_node ) , current_w_set ]
        current_level_list = [current_tuple]
        tree_dict['0'] = current_level_list
        prev_level_list = current_level_list
        for i in range(1,self.length+1): # 1,2,..,9
            # level in construction
            current_level = keys_list[i]
            print('current_level:' , current_level , 'prev_level_list length:' , len(prev_level_list))
            current_level_list = []
            for j in range(len(prev_level_list)):
                previous_node = prev_level_list[j]
                previous_node_node = previous_node[0]
                previous_w_set = previous_node[2]
                previous_outgoing = previous_node[1]
                for k in range(len(previous_outgoing)):
                    # create the nodes in the next level by applying transpositions in previous_outgoing to 
                    # previous_node_node and to previous_w_set
                    transposition = int(previous_outgoing[k][0])
                    current_node_node =  simple_transposition(previous_node_node , transposition)
                    current_w_set = apply_trasp_to_w(previous_w_set , transposition)
                    current_outgoing = outgoing_edges( current_node_node )
                    current_node = [current_node_node , current_outgoing , current_w_set]
                    current_level_list.append(current_node)
            # simplify current level list
            no_duplicate = []
            for e in range(len(current_level_list)):
                not_in_no_duplicate = True
                for h in range(len(no_duplicate)):
                    if mu_node_equality( no_duplicate[h][0] ,  current_level_list[e][0]):
                        not_in_no_duplicate = False
                        # record h here
                        change_index = h
                if not_in_no_duplicate:
                    no_duplicate.append(current_level_list[e] )
                else:
                    # update the w set of the node in no_uplicate to be the union of the two W-sets
                    no_duplicate[change_index][2] = no_duplicate[change_index][2].union(current_level_list[e][2])
            current_level_list = no_duplicate
            tree_dict[current_level] = current_level_list
            prev_level_list = current_level_list
        self.tree_dict = tree_dict
        self.invert_w_sets()
        return tree_dict[str(self.length)][0][2]

In [3]:
##############
## Examples ##
##############

In [4]:
## n = 2 ##
node1 = muInvolutionNode([2], [1,2], [[1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2]
number of levels: 0


{(1, 2)}

In [5]:
tree1.tree_dict

{'0': [[[1 2], [], {(1, 2)}]]}

In [6]:
## n = 4 ##
node1 = muInvolutionNode([4], [1,2,3,4], [[1,2,3,4]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4]
number of levels: 2
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 1


{(1, 4, 2, 3), (2, 3, 1, 4)}

In [7]:
tree1.tree_dict

{'0': [[[1 2 3 4], [(2, [1 3 2 4])], {(1, 2, 3, 4)}]],
 '1': [[[1 3 2 4], [(1, [1 4 2 3]), (3, [1 4 2 3])], {(1, 3, 2, 4)}]],
 '2': [[[1 4 2 3], [], {(1, 4, 2, 3), (2, 3, 1, 4)}]]}

In [8]:
node1 = muInvolutionNode([2,2], [1,2,3,4], [[1,2] , [1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4]
number of levels: 4
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 1
current_level: 3 prev_level_list length: 2
current_level: 4 prev_level_list length: 1


{(3, 4, 1, 2)}

In [9]:
tree1.tree_dict

{'0': [[[1 2|3 4], [(2, [1 3|2 4])], {(1, 2, 3, 4)}]],
 '1': [[[1 3|2 4], [(1, [2 3|1 4]), (3, [1 4|2 3])], {(1, 3, 2, 4)}]],
 '2': [[[2 3|1 4], [(3, [2 4|1 3])], {(2, 3, 1, 4)}],
  [[1 4|2 3], [(1, [2 4|1 3])], {(1, 4, 2, 3)}]],
 '3': [[[2 4|1 3], [(2, [3 4|1 2])], {(2, 4, 1, 3)}]],
 '4': [[[3 4|1 2], [], {(3, 4, 1, 2)}]]}

In [10]:
## n = 6 ##
node1 = muInvolutionNode([6], [1,2,3,4,5,6], [[1,2,3,4,5,6]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4 5 6]
number of levels: 6
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 2
current_level: 3 prev_level_list length: 3
current_level: 4 prev_level_list length: 3
current_level: 5 prev_level_list length: 3
current_level: 6 prev_level_list length: 2


{(1, 6, 2, 5, 3, 4),
 (1, 6, 3, 4, 2, 5),
 (2, 5, 1, 6, 3, 4),
 (2, 5, 3, 4, 1, 6),
 (3, 4, 1, 6, 2, 5),
 (3, 4, 2, 5, 1, 6)}

In [11]:
tree1.tree_dict

{'0': [[[1 2 3 4 5 6],
   [(2, [1 3 2 4 5 6]), (4, [1 2 3 5 4 6])],
   {(1, 2, 3, 4, 5, 6)}]],
 '1': [[[1 3 2 4 5 6],
   [(1, [1 4 2 3 5 6]), (3, [1 4 2 3 5 6]), (4, [1 3 2 5 4 6])],
   {(1, 3, 2, 4, 5, 6)}],
  [[1 2 3 5 4 6],
   [(2, [1 3 2 5 4 6]), (3, [1 2 3 6 4 5]), (5, [1 2 3 6 4 5])],
   {(1, 2, 3, 5, 4, 6)}]],
 '2': [[[1 4 2 3 5 6],
   [(4, [1 5 2 3 4 6])],
   {(1, 4, 2, 3, 5, 6), (2, 3, 1, 4, 5, 6)}],
  [[1 3 2 5 4 6],
   [(1, [1 5 2 3 4 6]), (3, [1 4 2 5 3 6]), (5, [1 3 2 6 4 5])],
   {(1, 3, 2, 5, 4, 6)}],
  [[1 2 3 6 4 5],
   [(2, [1 3 2 6 4 5])],
   {(1, 2, 3, 6, 4, 5), (1, 2, 4, 5, 3, 6)}]],
 '3': [[[1 5 2 3 4 6],
   [(3, [1 5 2 4 3 6]), (5, [1 6 2 3 4 5])],
   {(1, 5, 2, 3, 4, 6), (2, 3, 1, 5, 4, 6)}],
  [[1 4 2 5 3 6],
   [(1, [1 5 2 4 3 6]),
    (2, [1 4 2 6 3 5]),
    (4, [1 5 2 4 3 6]),
    (5, [1 4 2 6 3 5])],
   {(1, 4, 2, 5, 3, 6)}],
  [[1 3 2 6 4 5],
   [(1, [1 6 2 3 4 5]), (3, [1 4 2 6 3 5])],
   {(1, 3, 2, 6, 4, 5), (1, 3, 4, 5, 2, 6)}]],
 '4': [[[1 5 2 4 3 6],


In [12]:
node1 = muInvolutionNode([4,2], [1,2,3,4,5,6], [[1,2,3,4],[1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4|5 6]
number of levels: 10
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 2
current_level: 3 prev_level_list length: 4
current_level: 4 prev_level_list length: 5
current_level: 5 prev_level_list length: 7
current_level: 6 prev_level_list length: 7
current_level: 7 prev_level_list length: 7
current_level: 8 prev_level_list length: 5
current_level: 9 prev_level_list length: 4
current_level: 10 prev_level_list length: 2


{(3, 6, 4, 5, 1, 2), (4, 5, 3, 6, 1, 2)}

In [13]:
tree1.tree_dict

{'0': [[[1 2 3 4|5 6],
   [(2, [1 3 2 4|5 6]), (4, [1 2 3 5|4 6])],
   {(1, 2, 3, 4, 5, 6)}]],
 '1': [[[1 3 2 4|5 6],
   [(1, [1 4 2 3|5 6]), (3, [1 4 2 3|5 6]), (4, [1 3 2 5|4 6])],
   {(1, 3, 2, 4, 5, 6)}],
  [[1 2 3 5|4 6],
   [(2, [1 3 2 5|4 6]), (3, [1 2 4 5|3 6]), (5, [1 2 3 6|4 5])],
   {(1, 2, 3, 5, 4, 6)}]],
 '2': [[[1 4 2 3|5 6],
   [(4, [1 5 2 3|4 6])],
   {(1, 4, 2, 3, 5, 6), (2, 3, 1, 4, 5, 6)}],
  [[1 3 2 5|4 6],
   [(1, [1 5 2 3|4 6]), (3, [1 4 2 5|3 6]), (5, [1 3 2 6|4 5])],
   {(1, 3, 2, 5, 4, 6)}],
  [[1 2 4 5|3 6],
   [(2, [1 3 4 5|2 6]), (5, [1 2 4 6|3 5])],
   {(1, 2, 4, 5, 3, 6)}],
  [[1 2 3 6|4 5],
   [(2, [1 3 2 6|4 5]), (3, [1 2 4 6|3 5])],
   {(1, 2, 3, 6, 4, 5)}]],
 '3': [[[1 5 2 3|4 6],
   [(3, [1 5 2 4|3 6]), (5, [1 6 2 3|4 5])],
   {(1, 5, 2, 3, 4, 6), (2, 3, 1, 5, 4, 6)}],
  [[1 4 2 5|3 6],
   [(1, [1 5 2 4|3 6]),
    (2, [1 4 3 5|2 6]),
    (4, [1 5 2 4|3 6]),
    (5, [1 4 2 6|3 5])],
   {(1, 4, 2, 5, 3, 6)}],
  [[1 3 2 6|4 5],
   [(1, [1 6 2 3|4 5]), (3

In [14]:
node1 = muInvolutionNode([2,4], [1,2,3,4,5,6], [[1,2],[1,2,3,4]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4 5 6]
number of levels: 10
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 2
current_level: 3 prev_level_list length: 4
current_level: 4 prev_level_list length: 5
current_level: 5 prev_level_list length: 7
current_level: 6 prev_level_list length: 7
current_level: 7 prev_level_list length: 7
current_level: 8 prev_level_list length: 5
current_level: 9 prev_level_list length: 4
current_level: 10 prev_level_list length: 2


{(5, 6, 1, 4, 2, 3), (5, 6, 2, 3, 1, 4)}

In [15]:
tree1.tree_dict

{'0': [[[1 2|3 4 5 6],
   [(2, [1 3|2 4 5 6]), (4, [1 2|3 5 4 6])],
   {(1, 2, 3, 4, 5, 6)}]],
 '1': [[[1 3|2 4 5 6],
   [(1, [2 3|1 4 5 6]), (3, [1 4|2 3 5 6]), (4, [1 3|2 5 4 6])],
   {(1, 3, 2, 4, 5, 6)}],
  [[1 2|3 5 4 6],
   [(2, [1 3|2 5 4 6]), (3, [1 2|3 6 4 5]), (5, [1 2|3 6 4 5])],
   {(1, 2, 3, 5, 4, 6)}]],
 '2': [[[2 3|1 4 5 6],
   [(3, [2 4|1 3 5 6]), (4, [2 3|1 5 4 6])],
   {(2, 3, 1, 4, 5, 6)}],
  [[1 4|2 3 5 6],
   [(1, [2 4|1 3 5 6]), (4, [1 5|2 3 4 6])],
   {(1, 4, 2, 3, 5, 6)}],
  [[1 3|2 5 4 6],
   [(1, [2 3|1 5 4 6]), (3, [1 4|2 5 3 6]), (5, [1 3|2 6 4 5])],
   {(1, 3, 2, 5, 4, 6)}],
  [[1 2|3 6 4 5],
   [(2, [1 3|2 6 4 5])],
   {(1, 2, 3, 6, 4, 5), (1, 2, 4, 5, 3, 6)}]],
 '3': [[[2 4|1 3 5 6],
   [(2, [3 4|1 2 5 6]), (4, [2 5|1 3 4 6])],
   {(2, 4, 1, 3, 5, 6)}],
  [[2 3|1 5 4 6],
   [(3, [2 4|1 5 3 6]), (5, [2 3|1 6 4 5])],
   {(2, 3, 1, 5, 4, 6)}],
  [[1 5|2 3 4 6],
   [(1, [2 5|1 3 4 6]), (3, [1 5|2 4 3 6]), (5, [1 6|2 3 4 5])],
   {(1, 5, 2, 3, 4, 6)}],
  [[1 4

In [16]:
node1 = muInvolutionNode([2,2,2], [1,2,3,4,5,6], [[1,2],[1,2],[1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4|5 6]
number of levels: 12
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 2
current_level: 3 prev_level_list length: 5
current_level: 4 prev_level_list length: 7
current_level: 5 prev_level_list length: 11
current_level: 6 prev_level_list length: 12
current_level: 7 prev_level_list length: 14
current_level: 8 prev_level_list length: 12
current_level: 9 prev_level_list length: 11
current_level: 10 prev_level_list length: 7
current_level: 11 prev_level_list length: 5
current_level: 12 prev_level_list length: 2


{(5, 6, 3, 4, 1, 2)}

In [17]:
tree1.tree_dict

{'0': [[[1 2|3 4|5 6],
   [(2, [1 3|2 4|5 6]), (4, [1 2|3 5|4 6])],
   {(1, 2, 3, 4, 5, 6)}]],
 '1': [[[1 3|2 4|5 6],
   [(1, [2 3|1 4|5 6]), (3, [1 4|2 3|5 6]), (4, [1 3|2 5|4 6])],
   {(1, 3, 2, 4, 5, 6)}],
  [[1 2|3 5|4 6],
   [(2, [1 3|2 5|4 6]), (3, [1 2|4 5|3 6]), (5, [1 2|3 6|4 5])],
   {(1, 2, 3, 5, 4, 6)}]],
 '2': [[[2 3|1 4|5 6],
   [(3, [2 4|1 3|5 6]), (4, [2 3|1 5|4 6])],
   {(2, 3, 1, 4, 5, 6)}],
  [[1 4|2 3|5 6],
   [(1, [2 4|1 3|5 6]), (4, [1 5|2 3|4 6])],
   {(1, 4, 2, 3, 5, 6)}],
  [[1 3|2 5|4 6],
   [(1, [2 3|1 5|4 6]), (3, [1 4|2 5|3 6]), (5, [1 3|2 6|4 5])],
   {(1, 3, 2, 5, 4, 6)}],
  [[1 2|4 5|3 6],
   [(2, [1 3|4 5|2 6]), (5, [1 2|4 6|3 5])],
   {(1, 2, 4, 5, 3, 6)}],
  [[1 2|3 6|4 5],
   [(2, [1 3|2 6|4 5]), (3, [1 2|4 6|3 5])],
   {(1, 2, 3, 6, 4, 5)}]],
 '3': [[[2 4|1 3|5 6],
   [(2, [3 4|1 2|5 6]), (4, [2 5|1 3|4 6])],
   {(2, 4, 1, 3, 5, 6)}],
  [[2 3|1 5|4 6],
   [(3, [2 4|1 5|3 6]), (5, [2 3|1 6|4 5])],
   {(2, 3, 1, 5, 4, 6)}],
  [[1 5|2 3|4 6],
   [(1, [

In [18]:
node1 = muInvolutionNode([8], [1,2,3,4,5,6,7,8], [[1,2,3,4,5,6,7,8]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4 5 6 7 8]
number of levels: 12
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 6
current_level: 4 prev_level_list length: 9
current_level: 5 prev_level_list length: 12
current_level: 6 prev_level_list length: 14
current_level: 7 prev_level_list length: 15
current_level: 8 prev_level_list length: 14
current_level: 9 prev_level_list length: 12
current_level: 10 prev_level_list length: 9
current_level: 11 prev_level_list length: 6
current_level: 12 prev_level_list length: 3


{(1, 8, 2, 7, 3, 6, 4, 5),
 (1, 8, 2, 7, 4, 5, 3, 6),
 (1, 8, 3, 6, 2, 7, 4, 5),
 (1, 8, 3, 6, 4, 5, 2, 7),
 (1, 8, 4, 5, 2, 7, 3, 6),
 (1, 8, 4, 5, 3, 6, 2, 7),
 (2, 7, 1, 8, 3, 6, 4, 5),
 (2, 7, 1, 8, 4, 5, 3, 6),
 (2, 7, 3, 6, 1, 8, 4, 5),
 (2, 7, 3, 6, 4, 5, 1, 8),
 (2, 7, 4, 5, 1, 8, 3, 6),
 (2, 7, 4, 5, 3, 6, 1, 8),
 (3, 6, 1, 8, 2, 7, 4, 5),
 (3, 6, 1, 8, 4, 5, 2, 7),
 (3, 6, 2, 7, 1, 8, 4, 5),
 (3, 6, 2, 7, 4, 5, 1, 8),
 (3, 6, 4, 5, 1, 8, 2, 7),
 (3, 6, 4, 5, 2, 7, 1, 8),
 (4, 5, 1, 8, 2, 7, 3, 6),
 (4, 5, 1, 8, 3, 6, 2, 7),
 (4, 5, 2, 7, 1, 8, 3, 6),
 (4, 5, 2, 7, 3, 6, 1, 8),
 (4, 5, 3, 6, 1, 8, 2, 7),
 (4, 5, 3, 6, 2, 7, 1, 8)}

In [19]:
tree1.tree_dict

{'0': [[[1 2 3 4 5 6 7 8],
   [(2, [1 3 2 4 5 6 7 8]), (4, [1 2 3 5 4 6 7 8]), (6, [1 2 3 4 5 7 6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3 2 4 5 6 7 8],
   [(1, [1 4 2 3 5 6 7 8]),
    (3, [1 4 2 3 5 6 7 8]),
    (4, [1 3 2 5 4 6 7 8]),
    (6, [1 3 2 4 5 7 6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2 3 5 4 6 7 8],
   [(2, [1 3 2 5 4 6 7 8]),
    (3, [1 2 3 6 4 5 7 8]),
    (5, [1 2 3 6 4 5 7 8]),
    (6, [1 2 3 5 4 7 6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2 3 4 5 7 6 8],
   [(2, [1 3 2 4 5 7 6 8]),
    (4, [1 2 3 5 4 7 6 8]),
    (5, [1 2 3 4 5 8 6 7]),
    (7, [1 2 3 4 5 8 6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[1 4 2 3 5 6 7 8],
   [(4, [1 5 2 3 4 6 7 8]), (6, [1 4 2 3 5 7 6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8), (2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 3 2 5 4 6 7 8],
   [(1, [1 5 2 3 4 6 7 8]),
    (3, [1 4 2 5 3 6 7 8]),
    (5, [1 3 2 6 4 5 7 8]),
    (6, [1 3 2 5 4 7 6 8])],
   {(1, 3, 2, 5, 4, 6, 7, 8)}],
  [[1 3 2 4 5 7 6 8],
   [(1, [1 4 2 3 5 7 6 8]),
 

In [20]:
node1 = muInvolutionNode([4,4], [1,2,3,4,5,6,7,8], [[1,2,3,4],[1,2,3,4]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4|5 6 7 8]
number of levels: 20
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 7
current_level: 4 prev_level_list length: 12
current_level: 5 prev_level_list length: 20
current_level: 6 prev_level_list length: 29
current_level: 7 prev_level_list length: 40
current_level: 8 prev_level_list length: 49
current_level: 9 prev_level_list length: 58
current_level: 10 prev_level_list length: 63
current_level: 11 prev_level_list length: 66
current_level: 12 prev_level_list length: 63
current_level: 13 prev_level_list length: 58
current_level: 14 prev_level_list length: 49
current_level: 15 prev_level_list length: 40
current_level: 16 prev_level_list length: 29
current_level: 17 prev_level_list length: 20
current_level: 18 prev_level_list length: 12
current_level: 19 prev_level_list length: 7
current_level: 20 prev_level_list length: 3


{(5, 8, 6, 7, 1, 4, 2, 3),
 (5, 8, 6, 7, 2, 3, 1, 4),
 (6, 7, 5, 8, 1, 4, 2, 3),
 (6, 7, 5, 8, 2, 3, 1, 4)}

In [21]:
tree1.tree_dict

{'0': [[[1 2 3 4|5 6 7 8],
   [(2, [1 3 2 4|5 6 7 8]), (4, [1 2 3 5|4 6 7 8]), (6, [1 2 3 4|5 7 6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3 2 4|5 6 7 8],
   [(1, [1 4 2 3|5 6 7 8]),
    (3, [1 4 2 3|5 6 7 8]),
    (4, [1 3 2 5|4 6 7 8]),
    (6, [1 3 2 4|5 7 6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2 3 5|4 6 7 8],
   [(2, [1 3 2 5|4 6 7 8]),
    (3, [1 2 4 5|3 6 7 8]),
    (5, [1 2 3 6|4 5 7 8]),
    (6, [1 2 3 5|4 7 6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2 3 4|5 7 6 8],
   [(2, [1 3 2 4|5 7 6 8]),
    (4, [1 2 3 5|4 7 6 8]),
    (5, [1 2 3 4|5 8 6 7]),
    (7, [1 2 3 4|5 8 6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[1 4 2 3|5 6 7 8],
   [(4, [1 5 2 3|4 6 7 8]), (6, [1 4 2 3|5 7 6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8), (2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 3 2 5|4 6 7 8],
   [(1, [1 5 2 3|4 6 7 8]),
    (3, [1 4 2 5|3 6 7 8]),
    (5, [1 3 2 6|4 5 7 8]),
    (6, [1 3 2 5|4 7 6 8])],
   {(1, 3, 2, 5, 4, 6, 7, 8)}],
  [[1 3 2 4|5 7 6 8],
   [(1, [1 4 2 3|5 7 6 8]),
 

In [22]:
node1 = muInvolutionNode([2,2,4], [1,2,3,4,5,6,7,8], [[1,2],[1,2],[1,2,3,4]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4|5 6 7 8]
number of levels: 22
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 8
current_level: 4 prev_level_list length: 15
current_level: 5 prev_level_list length: 27
current_level: 6 prev_level_list length: 41
current_level: 7 prev_level_list length: 60
current_level: 8 prev_level_list length: 78
current_level: 9 prev_level_list length: 98
current_level: 10 prev_level_list length: 112
current_level: 11 prev_level_list length: 124
current_level: 12 prev_level_list length: 126
current_level: 13 prev_level_list length: 124
current_level: 14 prev_level_list length: 112
current_level: 15 prev_level_list length: 98
current_level: 16 prev_level_list length: 78
current_level: 17 prev_level_list length: 60
current_level: 18 prev_level_list length: 41
current_level: 19 prev_level_list length: 27
current_level: 20 prev_level_list length: 15
current_level: 21 prev_level_list length: 8
current_level: 22 prev_l

{(7, 8, 5, 6, 1, 4, 2, 3), (7, 8, 5, 6, 2, 3, 1, 4)}

In [23]:
tree1.tree_dict

{'0': [[[1 2|3 4|5 6 7 8],
   [(2, [1 3|2 4|5 6 7 8]), (4, [1 2|3 5|4 6 7 8]), (6, [1 2|3 4|5 7 6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3|2 4|5 6 7 8],
   [(1, [2 3|1 4|5 6 7 8]),
    (3, [1 4|2 3|5 6 7 8]),
    (4, [1 3|2 5|4 6 7 8]),
    (6, [1 3|2 4|5 7 6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2|3 5|4 6 7 8],
   [(2, [1 3|2 5|4 6 7 8]),
    (3, [1 2|4 5|3 6 7 8]),
    (5, [1 2|3 6|4 5 7 8]),
    (6, [1 2|3 5|4 7 6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2|3 4|5 7 6 8],
   [(2, [1 3|2 4|5 7 6 8]),
    (4, [1 2|3 5|4 7 6 8]),
    (5, [1 2|3 4|5 8 6 7]),
    (7, [1 2|3 4|5 8 6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[2 3|1 4|5 6 7 8],
   [(3, [2 4|1 3|5 6 7 8]), (4, [2 3|1 5|4 6 7 8]), (6, [2 3|1 4|5 7 6 8])],
   {(2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 4|2 3|5 6 7 8],
   [(1, [2 4|1 3|5 6 7 8]), (4, [1 5|2 3|4 6 7 8]), (6, [1 4|2 3|5 7 6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8)}],
  [[1 3|2 5|4 6 7 8],
   [(1, [2 3|1 5|4 6 7 8]),
    (3, [1 4|2 5|3 6 7 8]),
    (5, [1 

In [24]:
node1 = muInvolutionNode([4,2,2], [1,2,3,4,5,6,7,8], [[1,2,3,4],[1,2],[1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4|5 6|7 8]
number of levels: 22
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 8
current_level: 4 prev_level_list length: 15
current_level: 5 prev_level_list length: 27
current_level: 6 prev_level_list length: 41
current_level: 7 prev_level_list length: 60
current_level: 8 prev_level_list length: 78
current_level: 9 prev_level_list length: 98
current_level: 10 prev_level_list length: 112
current_level: 11 prev_level_list length: 124
current_level: 12 prev_level_list length: 126
current_level: 13 prev_level_list length: 124
current_level: 14 prev_level_list length: 112
current_level: 15 prev_level_list length: 98
current_level: 16 prev_level_list length: 78
current_level: 17 prev_level_list length: 60
current_level: 18 prev_level_list length: 41
current_level: 19 prev_level_list length: 27
current_level: 20 prev_level_list length: 15
current_level: 21 prev_level_list length: 8
current_level: 22 prev_l

{(5, 8, 6, 7, 3, 4, 1, 2), (6, 7, 5, 8, 3, 4, 1, 2)}

In [25]:
tree1.tree_dict

{'0': [[[1 2 3 4|5 6|7 8],
   [(2, [1 3 2 4|5 6|7 8]), (4, [1 2 3 5|4 6|7 8]), (6, [1 2 3 4|5 7|6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3 2 4|5 6|7 8],
   [(1, [1 4 2 3|5 6|7 8]),
    (3, [1 4 2 3|5 6|7 8]),
    (4, [1 3 2 5|4 6|7 8]),
    (6, [1 3 2 4|5 7|6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2 3 5|4 6|7 8],
   [(2, [1 3 2 5|4 6|7 8]),
    (3, [1 2 4 5|3 6|7 8]),
    (5, [1 2 3 6|4 5|7 8]),
    (6, [1 2 3 5|4 7|6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2 3 4|5 7|6 8],
   [(2, [1 3 2 4|5 7|6 8]),
    (4, [1 2 3 5|4 7|6 8]),
    (5, [1 2 3 4|6 7|5 8]),
    (7, [1 2 3 4|5 8|6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[1 4 2 3|5 6|7 8],
   [(4, [1 5 2 3|4 6|7 8]), (6, [1 4 2 3|5 7|6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8), (2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 3 2 5|4 6|7 8],
   [(1, [1 5 2 3|4 6|7 8]),
    (3, [1 4 2 5|3 6|7 8]),
    (5, [1 3 2 6|4 5|7 8]),
    (6, [1 3 2 5|4 7|6 8])],
   {(1, 3, 2, 5, 4, 6, 7, 8)}],
  [[1 3 2 4|5 7|6 8],
   [(1, [1 4 2 3|5 7|6 8]),
 

In [26]:
node1 = muInvolutionNode([2,4,2], [1,2,3,4,5,6,7,8], [[1,2],[1,2,3,4], [1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4 5 6|7 8]
number of levels: 22
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 8
current_level: 4 prev_level_list length: 15
current_level: 5 prev_level_list length: 27
current_level: 6 prev_level_list length: 41
current_level: 7 prev_level_list length: 60
current_level: 8 prev_level_list length: 78
current_level: 9 prev_level_list length: 98
current_level: 10 prev_level_list length: 112
current_level: 11 prev_level_list length: 124
current_level: 12 prev_level_list length: 126
current_level: 13 prev_level_list length: 124
current_level: 14 prev_level_list length: 112
current_level: 15 prev_level_list length: 98
current_level: 16 prev_level_list length: 78
current_level: 17 prev_level_list length: 60
current_level: 18 prev_level_list length: 41
current_level: 19 prev_level_list length: 27
current_level: 20 prev_level_list length: 15
current_level: 21 prev_level_list length: 8
current_level: 22 prev_l

{(7, 8, 3, 6, 4, 5, 1, 2), (7, 8, 4, 5, 3, 6, 1, 2)}

In [27]:
tree1.tree_dict

{'0': [[[1 2|3 4 5 6|7 8],
   [(2, [1 3|2 4 5 6|7 8]), (4, [1 2|3 5 4 6|7 8]), (6, [1 2|3 4 5 7|6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3|2 4 5 6|7 8],
   [(1, [2 3|1 4 5 6|7 8]),
    (3, [1 4|2 3 5 6|7 8]),
    (4, [1 3|2 5 4 6|7 8]),
    (6, [1 3|2 4 5 7|6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2|3 5 4 6|7 8],
   [(2, [1 3|2 5 4 6|7 8]),
    (3, [1 2|3 6 4 5|7 8]),
    (5, [1 2|3 6 4 5|7 8]),
    (6, [1 2|3 5 4 7|6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2|3 4 5 7|6 8],
   [(2, [1 3|2 4 5 7|6 8]),
    (4, [1 2|3 5 4 7|6 8]),
    (5, [1 2|3 4 6 7|5 8]),
    (7, [1 2|3 4 5 8|6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[2 3|1 4 5 6|7 8],
   [(3, [2 4|1 3 5 6|7 8]), (4, [2 3|1 5 4 6|7 8]), (6, [2 3|1 4 5 7|6 8])],
   {(2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 4|2 3 5 6|7 8],
   [(1, [2 4|1 3 5 6|7 8]), (4, [1 5|2 3 4 6|7 8]), (6, [1 4|2 3 5 7|6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8)}],
  [[1 3|2 5 4 6|7 8],
   [(1, [2 3|1 5 4 6|7 8]),
    (3, [1 4|2 5 3 6|7 8]),
    (5, [1 

In [28]:
node1 = muInvolutionNode([2,6], [1,2,3,4,5,6,7,8], [[1,2],[1,2,3,4,5,6]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4 5 6 7 8]
number of levels: 18
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 7
current_level: 4 prev_level_list length: 12
current_level: 5 prev_level_list length: 19
current_level: 6 prev_level_list length: 26
current_level: 7 prev_level_list length: 34
current_level: 8 prev_level_list length: 40
current_level: 9 prev_level_list length: 45
current_level: 10 prev_level_list length: 46
current_level: 11 prev_level_list length: 45
current_level: 12 prev_level_list length: 40
current_level: 13 prev_level_list length: 34
current_level: 14 prev_level_list length: 26
current_level: 15 prev_level_list length: 19
current_level: 16 prev_level_list length: 12
current_level: 17 prev_level_list length: 7
current_level: 18 prev_level_list length: 3


{(7, 8, 1, 6, 2, 5, 3, 4),
 (7, 8, 1, 6, 3, 4, 2, 5),
 (7, 8, 2, 5, 1, 6, 3, 4),
 (7, 8, 2, 5, 3, 4, 1, 6),
 (7, 8, 3, 4, 1, 6, 2, 5),
 (7, 8, 3, 4, 2, 5, 1, 6)}

In [29]:
tree1.tree_dict

{'0': [[[1 2|3 4 5 6 7 8],
   [(2, [1 3|2 4 5 6 7 8]), (4, [1 2|3 5 4 6 7 8]), (6, [1 2|3 4 5 7 6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3|2 4 5 6 7 8],
   [(1, [2 3|1 4 5 6 7 8]),
    (3, [1 4|2 3 5 6 7 8]),
    (4, [1 3|2 5 4 6 7 8]),
    (6, [1 3|2 4 5 7 6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2|3 5 4 6 7 8],
   [(2, [1 3|2 5 4 6 7 8]),
    (3, [1 2|3 6 4 5 7 8]),
    (5, [1 2|3 6 4 5 7 8]),
    (6, [1 2|3 5 4 7 6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2|3 4 5 7 6 8],
   [(2, [1 3|2 4 5 7 6 8]),
    (4, [1 2|3 5 4 7 6 8]),
    (5, [1 2|3 4 5 8 6 7]),
    (7, [1 2|3 4 5 8 6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[2 3|1 4 5 6 7 8],
   [(3, [2 4|1 3 5 6 7 8]), (4, [2 3|1 5 4 6 7 8]), (6, [2 3|1 4 5 7 6 8])],
   {(2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 4|2 3 5 6 7 8],
   [(1, [2 4|1 3 5 6 7 8]), (4, [1 5|2 3 4 6 7 8]), (6, [1 4|2 3 5 7 6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8)}],
  [[1 3|2 5 4 6 7 8],
   [(1, [2 3|1 5 4 6 7 8]),
    (3, [1 4|2 5 3 6 7 8]),
    (5, [1 

In [30]:
node1 = muInvolutionNode([6,2], [1,2,3,4,5,6,7,8], [[1,2,3,4,5,6], [1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2 3 4 5 6|7 8]
number of levels: 18
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 7
current_level: 4 prev_level_list length: 12
current_level: 5 prev_level_list length: 19
current_level: 6 prev_level_list length: 26
current_level: 7 prev_level_list length: 34
current_level: 8 prev_level_list length: 40
current_level: 9 prev_level_list length: 45
current_level: 10 prev_level_list length: 46
current_level: 11 prev_level_list length: 45
current_level: 12 prev_level_list length: 40
current_level: 13 prev_level_list length: 34
current_level: 14 prev_level_list length: 26
current_level: 15 prev_level_list length: 19
current_level: 16 prev_level_list length: 12
current_level: 17 prev_level_list length: 7
current_level: 18 prev_level_list length: 3


{(3, 8, 4, 7, 5, 6, 1, 2),
 (3, 8, 5, 6, 4, 7, 1, 2),
 (4, 7, 3, 8, 5, 6, 1, 2),
 (4, 7, 5, 6, 3, 8, 1, 2),
 (5, 6, 3, 8, 4, 7, 1, 2),
 (5, 6, 4, 7, 3, 8, 1, 2)}

In [31]:
tree1.tree_dict

{'0': [[[1 2 3 4 5 6|7 8],
   [(2, [1 3 2 4 5 6|7 8]), (4, [1 2 3 5 4 6|7 8]), (6, [1 2 3 4 5 7|6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3 2 4 5 6|7 8],
   [(1, [1 4 2 3 5 6|7 8]),
    (3, [1 4 2 3 5 6|7 8]),
    (4, [1 3 2 5 4 6|7 8]),
    (6, [1 3 2 4 5 7|6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2 3 5 4 6|7 8],
   [(2, [1 3 2 5 4 6|7 8]),
    (3, [1 2 3 6 4 5|7 8]),
    (5, [1 2 3 6 4 5|7 8]),
    (6, [1 2 3 5 4 7|6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2 3 4 5 7|6 8],
   [(2, [1 3 2 4 5 7|6 8]),
    (4, [1 2 3 5 4 7|6 8]),
    (5, [1 2 3 4 6 7|5 8]),
    (7, [1 2 3 4 5 8|6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[1 4 2 3 5 6|7 8],
   [(4, [1 5 2 3 4 6|7 8]), (6, [1 4 2 3 5 7|6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8), (2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 3 2 5 4 6|7 8],
   [(1, [1 5 2 3 4 6|7 8]),
    (3, [1 4 2 5 3 6|7 8]),
    (5, [1 3 2 6 4 5|7 8]),
    (6, [1 3 2 5 4 7|6 8])],
   {(1, 3, 2, 5, 4, 6, 7, 8)}],
  [[1 3 2 4 5 7|6 8],
   [(1, [1 4 2 3 5 7|6 8]),
 

In [32]:
node1 = muInvolutionNode([2,2,2,2], [1,2,3,4,5,6,7,8], [[1,2],[1,2],[1,2],[1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4|5 6|7 8]
number of levels: 24
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 3
current_level: 3 prev_level_list length: 9
current_level: 4 prev_level_list length: 18
current_level: 5 prev_level_list length: 35
current_level: 6 prev_level_list length: 56
current_level: 7 prev_level_list length: 87
current_level: 8 prev_level_list length: 119
current_level: 9 prev_level_list length: 158
current_level: 10 prev_level_list length: 190
current_level: 11 prev_level_list length: 222
current_level: 12 prev_level_list length: 238
current_level: 13 prev_level_list length: 248
current_level: 14 prev_level_list length: 238
current_level: 15 prev_level_list length: 222
current_level: 16 prev_level_list length: 190
current_level: 17 prev_level_list length: 158
current_level: 18 prev_level_list length: 119
current_level: 19 prev_level_list length: 87
current_level: 20 prev_level_list length: 56
current_level: 21 prev_level_list length: 35
current_level: 22

{(7, 8, 5, 6, 3, 4, 1, 2)}

In [33]:
tree1.tree_dict

{'0': [[[1 2|3 4|5 6|7 8],
   [(2, [1 3|2 4|5 6|7 8]), (4, [1 2|3 5|4 6|7 8]), (6, [1 2|3 4|5 7|6 8])],
   {(1, 2, 3, 4, 5, 6, 7, 8)}]],
 '1': [[[1 3|2 4|5 6|7 8],
   [(1, [2 3|1 4|5 6|7 8]),
    (3, [1 4|2 3|5 6|7 8]),
    (4, [1 3|2 5|4 6|7 8]),
    (6, [1 3|2 4|5 7|6 8])],
   {(1, 3, 2, 4, 5, 6, 7, 8)}],
  [[1 2|3 5|4 6|7 8],
   [(2, [1 3|2 5|4 6|7 8]),
    (3, [1 2|4 5|3 6|7 8]),
    (5, [1 2|3 6|4 5|7 8]),
    (6, [1 2|3 5|4 7|6 8])],
   {(1, 2, 3, 5, 4, 6, 7, 8)}],
  [[1 2|3 4|5 7|6 8],
   [(2, [1 3|2 4|5 7|6 8]),
    (4, [1 2|3 5|4 7|6 8]),
    (5, [1 2|3 4|6 7|5 8]),
    (7, [1 2|3 4|5 8|6 7])],
   {(1, 2, 3, 4, 5, 7, 6, 8)}]],
 '2': [[[2 3|1 4|5 6|7 8],
   [(3, [2 4|1 3|5 6|7 8]), (4, [2 3|1 5|4 6|7 8]), (6, [2 3|1 4|5 7|6 8])],
   {(2, 3, 1, 4, 5, 6, 7, 8)}],
  [[1 4|2 3|5 6|7 8],
   [(1, [2 4|1 3|5 6|7 8]), (4, [1 5|2 3|4 6|7 8]), (6, [1 4|2 3|5 7|6 8])],
   {(1, 4, 2, 3, 5, 6, 7, 8)}],
  [[1 3|2 5|4 6|7 8],
   [(1, [2 3|1 5|4 6|7 8]),
    (3, [1 4|2 5|3 6|7 8]),
    (5, [1 

In [34]:
node1 = muInvolutionNode([2,2,2,2,2], [1,2,3,4,5,6,7,8,9,10], [[1,2],[1,2],[1,2],[1,2],[1,2]])
tree1 = inv_Tree(node1)
print(tree1.root)
tree1.construct_dictionary()

[1 2|3 4|5 6|7 8|9 10]
number of levels: 40
current_level: 1 prev_level_list length: 1
current_level: 2 prev_level_list length: 4
current_level: 3 prev_level_list length: 14
current_level: 4 prev_level_list length: 35
current_level: 5 prev_level_list length: 80
current_level: 6 prev_level_list length: 157
current_level: 7 prev_level_list length: 289
current_level: 8 prev_level_list length: 485
current_level: 9 prev_level_list length: 775
current_level: 10 prev_level_list length: 1160
current_level: 11 prev_level_list length: 1668
current_level: 12 prev_level_list length: 2279
current_level: 13 prev_level_list length: 3008
current_level: 14 prev_level_list length: 3804
current_level: 15 prev_level_list length: 4664
current_level: 16 prev_level_list length: 5507
current_level: 17 prev_level_list length: 6319
current_level: 18 prev_level_list length: 7004


KeyboardInterrupt: 